In [1]:
import os
import subprocess
import importlib
import pandas as pd
import tokenize
from matplotlib import pyplot as plt

In [2]:
import create_data_set_utils
importlib.reload(create_data_set_utils)

<module 'create_data_set_utils' from '/home/erez/research/code_completion/create_data_set_utils.py'>

In [3]:
dir_path = r"/home/erez/github_repos/siva/latest/"

In [ ]:
# get_data_set.unpack_siva_files_and_checkout_git_repos(dir_path)

In [ ]:
all_python_files_list = create_data_set_utils.get_python_files_paths(dir_path)

In [ ]:
len(all_python_files_list)

In [ ]:
python_files_df = create_data_set_utils.create_python_files_df(all_python_files_list)

In [ ]:
python_files_df.head()

In [ ]:
python_files_df['number_of_lines'].sum()

In [ ]:
tokens_df = create_data_set_utils.create_tokens_df(python_files_df)
tokens_df.head()

In [ ]:
tokens_df.shape

In [ ]:
tokens_df.memory_usage(deep=True).sum()

In [ ]:
len(tokens_df['string'].unique())

In [ ]:
tokens_df['type_name'].unique().tolist()

In [ ]:
tokens_df.groupby('type_name').size()

In [ ]:
tokens_df.groupby('type_name').apply(lambda x: len(x['string'].unique()))

In [ ]:
tokens_df[tokens_df['type_name'] == 'NEWLINE']['string'].value_counts()

In [ ]:
tokens_df[tokens_df['type_name'] == 'OP']['string'].value_counts()

In [ ]:
tokens_df[tokens_df['type_name'] == 'BACKQUOTE']['string'].value_counts()

In [ ]:
tokens_df[tokens_df['type_name'] == 'INDENT']['string'].value_counts()

In [ ]:
tokens_df[tokens_df['type_name'] == 'DEDENT']['string'].value_counts()

In [ ]:
tokens_df[tokens_df['type_name'] == 'NL']['string'].value_counts()

# read / load from csv

In [4]:
tokens_csv_file_path = 'tokens.csv'
python_files_csv_file_path = 'python_files.csv'
# python_files_df.to_csv(python_files_csv_file_path)
# tokens_df.to_csv(tokens_csv_file_path)
python_files_df = pd.read_csv(python_files_csv_file_path)
tokens_df = pd.read_csv(tokens_csv_file_path)

# imports

In [5]:
imports_df = tokens_df[tokens_df.line.str.startswith('import ') | tokens_df.line.str.startswith('from ')]
imports_df['line'] = imports_df['line'].str.strip()
imports_df.shape

/home/erez/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


(326359, 8)

In [6]:
imports_df.head()

,Unnamed: 0,end,file_path,line,line_index,start,string,type_name
1,1,"(1, 6)",/home/erez/github_repos/siva/latest/e2/temp_gi...,import sys,1,"(1, 0)",import,NAME
2,2,"(1, 10)",/home/erez/github_repos/siva/latest/e2/temp_gi...,import sys,1,"(1, 7)",sys,NAME
3,3,"(1, 11)",/home/erez/github_repos/siva/latest/e2/temp_gi...,import sys,1,"(1, 10)",\n,NEWLINE
4,4,"(2, 6)",/home/erez/github_repos/siva/latest/e2/temp_gi...,import os,2,"(2, 0)",import,NAME
5,5,"(2, 9)",/home/erez/github_repos/siva/latest/e2/temp_gi...,import os,2,"(2, 7)",os,NAME


In [7]:
imports_lines_df = imports_df.drop_duplicates(['file_path', 'line'])
imports_lines_df.shape

(56395, 8)

In [8]:
imports_counts_series = imports_lines_df.line.value_counts()
imports_counts_series.shape

(9026,)

In [9]:
imports_counts_df = pd.DataFrame(imports_counts_series)
imports_counts_df.rename(columns={'line': 'count'}, inplace=True)
imports_counts_df['line'] = imports_counts_df.index.str.strip()
imports_counts_df.reset_index(drop=True).head()

,count,line
0,3474,import numpy as np
1,3421,import tensorflow as tf
2,2803,from __future__ import print_function
3,2522,from __future__ import division
4,2455,from __future__ import absolute_import


In [10]:
imports_counts_df['is_import'] = imports_counts_df.line.str.startswith('import')

In [11]:
imports_counts_df.groupby('is_import')['count'].sum()

is_import
False    29848
True     26547
Name: count, dtype: int64

In [12]:
imports_counts_df.head()

,count,line,is_import
import numpy as np,3474,import numpy as np,True
import tensorflow as tf,3421,import tensorflow as tf,True
from __future__ import print_function,2803,from __future__ import print_function,False
from __future__ import division,2522,from __future__ import division,False
from __future__ import absolute_import,2455,from __future__ import absolute_import,False


In [13]:
def get_start_char(line):
    if line.startswith('import'):
        return line[len('import ')]
    else:
        return line[len('from ')]

In [14]:
imports_counts_df['start_char'] = imports_counts_df['line'].apply(get_start_char)

In [15]:
top_1_count = imports_counts_df.groupby(['is_import', 'start_char'])['count'].max()
top_1_precision = top_1_count.sum() / imports_counts_df['count'].sum()
top_1_precision

0.34215799272985192

In [22]:
def complete_code(string_to_complete, numer_of_options=3):
    relevant_df = imports_counts_df[imports_counts_df['line'].str.startswith(string_to_complete)]
    total_relevant = relevant_df['count'].sum()
    most_relevant_df = relevant_df.sort_values(by='count', ascending=False).iloc[:numer_of_options]
    results_df = most_relevant_df[['count', 'line']].reset_index(drop=True)
    results_df['prob'] = results_df['count'] / total_relevant
    print(results_df['prob'].sum())
    return results_df

In [29]:
complete_code('from _', 5)

0.990440251572


,count,line,prob
0,2803,from __future__ import print_function,0.352579
1,2522,from __future__ import division,0.317233
2,2455,from __future__ import absolute_import,0.308805
3,66,from __future__ import unicode_literals,0.008302
4,28,"from __future__ import absolute_import, divisi...",0.003522


In [45]:
def get_largest(g, n=3):
    return g.nlargest(n, 'count')['count'].sum()

In [50]:
top_n_count = imports_counts_df.groupby(['is_import', 'start_char']).apply(get_largest, 3).sum()
top_n_precision = top_n_count.sum() / imports_counts_df['count'].sum()
top_n_precision

0.55526199131128651